In [78]:
import pandas as pd
import seaborn as sns
import numpy as np

In [79]:
raw_data = pd.read_json("pizza_data.json")
df = pd.DataFrame(raw_data)
df.head()

,Size,Cheese,Sauce,Crust,Popularity,Price,Comments,Pizza_ID,topping_1,topping_2,topping_3
0,Large,Yes,No,Thin Crust,3 Stars,21.42,Sample comment,0,Pineapple,Spinach,Mushrooms
1,Medium,Yes,No,Sicilian,3 Stars,26.40,Sample comment,1,Hot Oil,None,None
2,Medium,Yes,No,Greek Style,4 Stars,16.56,Sample comment,2,Onions,Mushrooms,Sausage
3,Small,Yes,No,Deep Dish,3 Stars,24.29,Sample comment,3,Black Olives,Onions,Bacon
4,Small,Yes,Yes,Deep Dish,2 Stars,19.58,Sample comment,4,Perpperoni,None,None


In [80]:
 pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [82]:
features = ['Size', 'Cheese', 'Sauce', 'Crust', 'Popularity', 'Price', 'Pizza_ID', 'topping_1', 'topping_2', 'topping_3']

X = df[features]
X_encoded = pd.get_dummies(X)

nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X_encoded)
distances, indices  = nbrs.kneighbors(X_encoded)

print(indices[5])

[5 4 8 2]


In [83]:
def getNeighbors(pizza_id):
    neighbors = indices[pizza_id]
    new_neighbors = np.delete(neighbors, 0)
    print(new_neighbors)
    result = df.query('Pizza_ID in @new_neighbors')
    result.reset_index()
    display(result)
    json_file = "recs_results.json"
    result.to_json(json_file, orient="records")
        
getNeighbors(56)


[58 61 53]


,Size,Cheese,Sauce,Crust,Popularity,Price,Comments,Pizza_ID,topping_1,topping_2,topping_3
53,Large,No,No,Greek Style,5 Stars,25.03,Sample comment,53,Spinach,Onions,Extra Cheese
58,Small,No,Yes,Thick Crust,2 Stars,25.15,Sample comment,58,None,None,None
61,Extra Large,No,No,Thin Crust,5 Stars,27.49,Sample comment,61,Extra Cheese,Onions,Green Peppers


In [84]:
import pickle

# Specify the file path to save the pipeline
file_path = 'trained_model.pkl'

# Save the pipeline to disk
with open(file_path, 'wb') as file:
    pickle.dump(nbrs, file)

In [85]:
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)

distances, indices  = model.kneighbors(X_encoded)

